<a href="https://colab.research.google.com/github/bjarkedc/RAG_FLAN_T5/blob/main/t5_flan_rag_finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
!pip install rank-bm25 nltk transformers sentencepiece

In [20]:
import pandas as pd
import re
import string
import nltk

from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5TokenizerFast

nltk.download('punkt')
nltk.download('stopwords')
lm = "drive/MyDrive/data-anlp/flant5-base-starwars/" # change to appropriate drive
finetuned_model = T5ForConditionalGeneration.from_pretrained(lm)
tokenizer_fine_tuned = T5TokenizerFast.from_pretrained(lm)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
def parse_docs(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()

    # Regular expression to match <doc> elements
    doc_pattern = re.compile(r'<doc id="([^"]+)" url="([^"]+)" title="([^"]+)">(.*?)</doc>', re.DOTALL)

    # Find all matches
    matches = doc_pattern.findall(file_content)

    # Extract data and create a list of dictionaries
    docs = [{'id': match[0], 'url': match[1], 'title': match[2], 'text': match[3].strip()} for match in matches]
    return docs

# Create a DataFrame
def create_dataframe(docs):
    return pd.DataFrame(docs)

# Preprocess text: tokenize and remove stopwords - preprocess is the problem
def preprocess(text):
    # Define a set of special characters
    special_chars = set(string.punctuation)

    # Initialize the stemmer
    stemmer = PorterStemmer()

    # Load set of English stopwords
    stop_words = set(stopwords.words('english'))

    # Tokenize and convert to lowercase
    tokens = word_tokenize(text.lower())

    # Preprocess tokens
    processed_tokens = []
    for token in tokens:
        if token not in stop_words and not all(char in special_chars for char in token):
            stemmed_token = stemmer.stem(token)
            processed_tokens.append(stemmed_token)

    return processed_tokens


In [ ]:

file_path = 'drive/MyDrive/data-anlp/starwarsfandomcom-removed-space-and.txt'  # Replace with the actual path to your file
docs = parse_docs(file_path)
df = create_dataframe(docs)
df['tokenized_text'] = df['text'].apply(preprocess)
df = df[df['title'] != df['text']]

In [26]:
def query_bm25(query, num_results=50):
    query_tokens = preprocess(query)
    print(query_tokens)
    doc_scores = bm25.get_scores(query_tokens)
    top_doc_indices = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:num_results]
    return df.iloc[top_doc_indices]['text'].str.cat(sep=' ')

def generate_response(query, context):
    # Concatenate the query and context
    input_text = f'Answer this question: "{query}". Based off the following context: "{context}".'

    # Tokenize the input text
    input_ids = tokenizer_fine_tuned.encode(input_text, return_tensors='pt')

    # Truncate the input to the model's max length if necessary
    max_length = tokenizer_fine_tuned.model_max_length
    if input_ids.size(1) > max_length:
        input_ids = input_ids[:, :max_length]

    truncated_input_text = tokenizer_fine_tuned.decode(input_ids[0], skip_special_tokens=True)

    output_ids = finetuned_model.generate(input_ids)[0]

    response = tokenizer_fine_tuned.decode(output_ids, skip_special_tokens=True)

    return response, context, truncated_input_text


In [27]:
text_set_path = '/content/drive/MyDrive/data-anlp/star_wars_dataset_dev/' # add appropriate path with datasets
test_set_path = '/content/drive/MyDrive/data-anlp/star_wars_dataset_test/' # add appropriate path with datasets

# Function to read a file and return a list of lines
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file]

def robs_eval(gold, pred):
    """
    An answer is considered correct if at least half of the gold
    tokens are in the prediction. Note that this is a shortcut,
    and will favor long answers.
    """
    gold = set(gold.strip().lower().replace('.', '').split(' '))
    pred = set(pred.strip().lower().replace('.', '').split(' '))
    return len(gold.intersection(pred)) >= len(gold)/2


# Load the data
dev_questions = read_file_to_list(text_set_path + 'questions.txt')
dev_answers = read_file_to_list(text_set_path + 'answers.txt')

test_questions = read_file_to_list(test_set_path + 'questions.txt')
test_answers = read_file_to_list(test_set_path + 'answers.txt')





In [ ]:
### grid search
from sklearn.model_selection import ParameterGrid
import numpy as np
from tqdm import tqdm

max_pairs = len(dev_questions)

param_grid = {
    'k1': np.linspace(0.75, 1.75, num=3),
    'b': np.linspace(0.3, 0.7, num=3)
}
grid = ParameterGrid(param_grid)
tokenized_corpus = df['tokenized_text'].tolist()
best_score = -1
best_params = None
results = {}  # Dictionary to store results

for params in tqdm(grid, desc="Grid Search"):
    print(f"\n{params}")
    bm25 = BM25Okapi(corpus=tokenized_corpus, k1=params['k1'], b=params['b'])
    total_correct = 0

    for query, answer in tqdm(zip(dev_questions[:max_pairs], dev_answers[:max_pairs]), total=len(dev_questions[:max_pairs]), desc="Evaluating Queries"):
        context = query_bm25(query, 15)
        response, context, truncated_input_text = generate_response(query, context)
        print(f"""
{query}
{response}
{answer}
----------------------------------------------""")

        if robs_eval(answer, response):
            total_correct += 1
            print("total = " + str(total_correct))

    # Save the results
    results[str(params)] = total_correct

    # Assess the effectiveness of the parameters
    print(f"Total Correct: {total_correct}")
    if total_correct > best_score:
        best_score = total_correct
        best_params = params

print(f"Best Params: {best_params}, Best Score: {best_score}")
# Print the results dictionary
print("\nAll Results:")
for param, score in results.items():
    print(f"{param}: {score}")

Grid Search:   0%|          | 0/9 [00:00<?, ?it/s]


{'b': 0.3, 'k1': 0.75}



Evaluating Queries:   0%|          | 0/154 [00:00<?, ?it/s]

['true', 'fals', 'r2-d2', 'origin', 'dialogu', 'actual', 'speak', 'part']
20.838517823542407
[23350, 24805, 116423, 141969, 76057, 14792, 125585, 71247, 32869, 102514, 38108, 17648, 130491, 104548, 54311]


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(

Evaluating Queries:   1%|          | 1/154 [00:05<13:28,  5.28s/it]

...


What is Finn's stormtrooper number?
2187
FN-2187
----------------------------------------------
['rey', "'s", 'kylo', "'s", 'mind', 'connect']
28.888598308572277
[38767, 73163, 73166, 46035, 145762, 148108, 46176, 140426, 46423, 106931, 81219, 95594, 42973, 106918, 71444]



Evaluating Queries:  99%|█████████▉| 153/154 [11:06<00:04,  4.69s/it]


Why are Rey's and Kylo's minds connected?
Kylo Ren and Rey shared a unique Force-bond
Supreme Leader Snoke connected them.
----------------------------------------------
['rey', 'relat', 'palpatin']
15.105903075122551
[95594, 24569, 3675, 106921, 36702, 33722, 130286, 33202, 106927, 34635, 106919, 126823, 130485, 109399, 119862]



Grid Search: 100%|██████████| 9/9 [2:15:51<00:00, 905.75s/it]


How is Rey related to Palpatine?
Son
She's his granddaughter.
----------------------------------------------
Total Correct: 24
Best Params: {'b': 0.3, 'k1': 1.75}, Best Score: 29

All Results:
{'b': 0.3, 'k1': 0.75}: 25
{'b': 0.3, 'k1': 1.25}: 28
{'b': 0.3, 'k1': 1.75}: 29
{'b': 0.5, 'k1': 0.75}: 25
{'b': 0.5, 'k1': 1.25}: 27
{'b': 0.5, 'k1': 1.75}: 28
{'b': 0.7, 'k1': 0.75}: 25
{'b': 0.7, 'k1': 1.25}: 28
{'b': 0.7, 'k1': 1.75}: 24


In [ ]:
test_set_path = '/content/drive/MyDrive/data-anlp/star_wars_dataset_test/'
test_questions = read_file_to_list(test_set_path + 'questions.txt')
test_answers = read_file_to_list(test_set_path + 'answers.txt')

bm25 = BM25Okapi(corpus=df['tokenized_text'].tolist(), k1=1.75, b=0.3)

data = []
total_correct = 0

for query, answer in tqdm(zip(test_questions, test_answers), total=len(test_answers), desc="Processing"):
    context = query_bm25(query, 15)

    response, context, truncated_input_text = generate_response(query, context)

    # Print the text (optional, can be removed if not needed)
    print(f"""
{truncated_input_text}
{response}
{answer}
----------------------------------------------""")

    # Check if the response is correct
    if robs_eval(answer, response):
        total_correct += 1
        print("total = " + str(total_correct))

    data.append({'Query': query, 'Correct Answer': answer, 'Model Response': response})

df_results = pd.DataFrame(data)
print(f"Total correct answers: {total_correct}/{len(test_answers)}")


# Make context list and then generate test-based on that

In [13]:
file_path = '/content/drive/MyDrive/data-anlp/contextfile.txt'
list_from_file = []
with open(file_path, 'r') as file:
    for line in file:
        list_from_file.append(line.strip())

In [ ]:
from tqdm import tqdm
from transformers import set_seed
test_set_path = '/content/drive/MyDrive/data-anlp/star_wars_dataset_test/'
test_questions = read_file_to_list(test_set_path + 'questions.txt')
test_answers = read_file_to_list(test_set_path + 'answers.txt')

data = []
total_correct = 0

# Change generator in order to make sure the context is the same
def generate_response_context(context):
    # Concatenate the query and context
    input_text = context

    # Tokenize the input text
    input_ids = tokenizer_fine_tuned.encode(input_text, return_tensors='pt')

    # Truncate the input to the model's max length if necessary
    max_length = tokenizer_fine_tuned.model_max_length
    if input_ids.size(1) > max_length:
        input_ids = input_ids[:, :max_length]

    truncated_input_text = tokenizer_fine_tuned.decode(input_ids[0], skip_special_tokens=True)
    output_ids = finetuned_model.generate(input_ids)[0]
    response = tokenizer_fine_tuned.decode(output_ids, skip_special_tokens=True)

    return response, context, truncated_input_text

for query, answer, context in tqdm(zip(test_questions, test_answers, list_from_file), total=len(test_answers), desc="Processing"):


    response, context, truncated_input_text = generate_response_context(context)

    # Print the text (optional, can be removed if not needed)
    print(f"""
{truncated_input_text}
{response}
{answer}
----------------------------------------------""")

    # Check if the response is correct
    if robs_eval(answer, response):
        total_correct += 1
        print("total = " + str(total_correct))

    data.append({'Query': query, 'Context': truncated_input_text , 'Correct Answer': answer, 'Model Response': response})

df_results = pd.DataFrame(data)
print(f"Total correct answers: {total_correct}/{len(test_answers)}")